# Импорт библиотек

In [393]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [394]:
import pandas as pd

In [395]:
!pip install catboost

In [396]:
import numpy as np

from sklearn.model_selection import train_test_split
import torch

from tqdm import tqdm, trange, tqdm_notebook

In [397]:
import random

In [398]:
from sklearn.metrics import classification_report

In [399]:
from sklearn.metrics import classification_report, accuracy_score, balanced_accuracy_score
from sklearn.metrics import explained_variance_score, r2_score

In [400]:
#фиксируем seed
def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True

RANDOM_SEED = 1500

init_random_seed(value=RANDOM_SEED)

# Данные

In [401]:
dataset = pd.read_excel('/content/drive/MyDrive/Гилфорд Маг Диплом Розовская/New results/logits 34 Guilford.xlsx')

In [402]:
dataset = dataset.drop(columns=['Unnamed: 0.1','Unnamed: 0','ID', 'attempt', 'n', 'answer'])
dataset.head(5)

,origNew,simple_category,prepr_text,orig_v1,orig_v2,orig_v3,cat_v1,cat_v2,cat_v3,cat_v4
0,0,1,сложить из спичек домик,3.681304,-2.685261,-2.385877,-1.189256,4.626387,-1.855205,-1.131889
1,2,2,использовать спички как опору чего нибудь,-2.919616,-1.976754,5.205158,-1.326608,-1.099055,3.938310,-1.653843
2,0,2,спичками мжно ковыряться в зубах,4.041395,-2.963273,-2.460635,-1.020064,-1.630299,4.506532,-1.679100
3,2,2,если сложить спички в виде колодца можно испол...,-3.026895,-1.863899,5.181743,-2.341077,0.076745,3.568159,-1.647687
4,0,1,если разукрасить спички то можно сложить их на...,3.958443,-2.817475,-2.532558,-1.583953,4.732340,-1.147014,-1.611131


Тот столбик, который предсказываем (origNew или simple_category) лучше указать как int, оставшийся - как str

In [403]:
# для файла 69
'''dataset = dataset.astype({'origNew': int, 'simple_category': str, 'prepr_text': str,
                       'orig_v1': float, 'orig_v2': float, 'orig_v3': float,
                          'orig_v4': float, 'orig_v5': float, 'orig_v6': float,
                       'cat_v1': float, 'cat_v2': float, 'cat_v3': float, 'cat_v4': float,
                          'cat_v5': float, 'cat_v6': float, 'cat_v7': float, 'cat_v8': float, 'cat_v9': float})'''

"dataset = dataset.astype({'origNew': int, 'simple_category': str, 'prepr_text': str,\n                       'orig_v1': float, 'orig_v2': float, 'orig_v3': float,\n                          'orig_v4': float, 'orig_v5': float, 'orig_v6': float,\n                       'cat_v1': float, 'cat_v2': float, 'cat_v3': float, 'cat_v4': float,\n                          'cat_v5': float, 'cat_v6': float, 'cat_v7': float, 'cat_v8': float, 'cat_v9': float})"

In [404]:
# для файла 34
dataset = dataset.astype({'origNew': int, 'simple_category': str, 'prepr_text': str,
                       'orig_v1': float, 'orig_v2': float, 'orig_v3': float,
                       'cat_v1': float, 'cat_v2': float, 'cat_v3': float, 'cat_v4': float})

In [405]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6855 entries, 0 to 6854
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   origNew          6855 non-null   int64  
 1   simple_category  6855 non-null   object 
 2   prepr_text       6855 non-null   object 
 3   orig_v1          6855 non-null   float64
 4   orig_v2          6855 non-null   float64
 5   orig_v3          6855 non-null   float64
 6   cat_v1           6855 non-null   float64
 7   cat_v2           6855 non-null   float64
 8   cat_v3           6855 non-null   float64
 9   cat_v4           6855 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 535.7+ KB


In [406]:
# для предсказания оригинальности
'''train_data, eval_data, train_labels, eval_labels = train_test_split(dataset.drop(['origNew'], axis=1), dataset.origNew,
                                                                    test_size=0.2, stratify=dataset.origNew, random_state=RANDOM_SEED)'''

"train_data, eval_data, train_labels, eval_labels = train_test_split(dataset.drop(['origNew'], axis=1), dataset.origNew,\n                                                                    test_size=0.2, stratify=dataset.origNew, random_state=RANDOM_SEED)"

In [407]:
# для предсказания категорий
train_data, eval_data, train_labels, eval_labels = train_test_split(dataset.drop(['simple_category'], axis=1), dataset.simple_category,
                                                                    test_size=0.2, stratify=dataset.simple_category, random_state=RANDOM_SEED)

In [408]:
# Initialize data
cat_features = [0]
text_features = [1]

In [409]:
train_data = train_data.values.tolist()
eval_data = eval_data.values.tolist()
train_labels = train_labels.values.tolist()
eval_labels = eval_labels.values.tolist()

# Модель 1: CB Classifier

In [410]:
from catboost import CatBoostClassifier

In [411]:
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=500,
                           learning_rate=1,
                           depth=7)

In [412]:
# Fit model
fit_model = model.fit(train_data,
          train_labels,
          cat_features = cat_features,
          text_features = text_features,
          eval_set=(eval_data, eval_labels))

0:	learn: 0.3002874	test: 0.5241007	best: 0.5241007 (0)	total: 2.16s	remaining: 17m 58s
1:	learn: 0.2439098	test: 0.4992992	best: 0.4992992 (1)	total: 4.43s	remaining: 18m 22s
2:	learn: 0.2355296	test: 0.5055052	best: 0.4992992 (1)	total: 6.13s	remaining: 16m 55s
3:	learn: 0.2332602	test: 0.5001279	best: 0.4992992 (1)	total: 8.1s	remaining: 16m 44s
4:	learn: 0.2238635	test: 0.4965307	best: 0.4965307 (4)	total: 10.1s	remaining: 16m 39s
5:	learn: 0.2209024	test: 0.4994117	best: 0.4965307 (4)	total: 12.3s	remaining: 16m 54s
6:	learn: 0.2159278	test: 0.5045555	best: 0.4965307 (4)	total: 14.4s	remaining: 16m 52s
7:	learn: 0.2110596	test: 0.5041882	best: 0.4965307 (4)	total: 17.6s	remaining: 18m 1s
8:	learn: 0.2080689	test: 0.5091092	best: 0.4965307 (4)	total: 20.2s	remaining: 18m 20s
9:	learn: 0.2063683	test: 0.5047415	best: 0.4965307 (4)	total: 22.5s	remaining: 18m 20s
10:	learn: 0.2015389	test: 0.4969075	best: 0.4965307 (4)	total: 24.8s	remaining: 18m 22s
11:	learn: 0.2003367	test: 0.4954

In [413]:
# Save model
fit_model.save_model('/content/drive/MyDrive/Гилфорд Маг Диплом Розовская/New results/CB2_class_cat_4_500-1-7')

In [414]:
# Load model
model = CatBoostClassifier()
fit_model1 = model.load_model('/content/drive/MyDrive/Гилфорд Маг Диплом Розовская/New results/CB2_class_cat_4_500-1-7')

In [415]:
# Get predicted classes
preds_class = fit_model1.predict(eval_data)
# Get predicted probabilities for each class
preds_proba = fit_model1.predict_proba(eval_data)
# Get predicted RawFormulaVal
preds_raw = fit_model1.predict(eval_data, prediction_type='RawFormulaVal')

In [416]:
print('accuracy: ', accuracy_score(eval_labels, preds_class))
print('balanced_accuracy: ', balanced_accuracy_score(eval_labels, preds_class))

accuracy:  0.8781911013858498
balanced_accuracy:  0.8593721056069974


In [417]:
print('get_best_iteration: ', fit_model1.get_best_iteration())
print('get_best_score: ', fit_model1.get_best_score())

get_best_iteration:  109
get_best_score:  {'learn': {'MultiClass': 0.016270384318347472}, 'validation': {'MultiClass': 0.45628508419301406}}


In [418]:
print(classification_report(eval_labels, preds_class))

              precision    recall  f1-score   support

           0       0.88      0.85      0.86       178
           1       0.85      0.90      0.87       353
           2       0.90      0.91      0.91       607
           3       0.87      0.78      0.82       233

    accuracy                           0.88      1371
   macro avg       0.87      0.86      0.87      1371
weighted avg       0.88      0.88      0.88      1371



# Модель 2: CB Regressor

In [358]:
from catboost import CatBoostRegressor, Pool

In [359]:
from sklearn.metrics import explained_variance_score, r2_score

In [360]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=500,
                          learning_rate=0.8,
                          depth=10)

In [361]:
# Fit model
fit_model = model.fit(train_data,
                       train_labels,
                       cat_features = cat_features,
                       text_features = text_features,
                       eval_set=(eval_data, eval_labels))

0:	learn: 0.5597094	test: 0.6168326	best: 0.6168326 (0)	total: 1.7s	remaining: 14m 9s
1:	learn: 0.5188807	test: 0.5905717	best: 0.5905717 (1)	total: 3.64s	remaining: 15m 5s
2:	learn: 0.5111947	test: 0.5852179	best: 0.5852179 (2)	total: 5.58s	remaining: 15m 24s
3:	learn: 0.5065796	test: 0.5865688	best: 0.5852179 (2)	total: 7.37s	remaining: 15m 14s
4:	learn: 0.5039827	test: 0.5857261	best: 0.5852179 (2)	total: 9.1s	remaining: 15m
5:	learn: 0.5009786	test: 0.5863657	best: 0.5852179 (2)	total: 11.5s	remaining: 15m 44s
6:	learn: 0.4956524	test: 0.5843252	best: 0.5843252 (6)	total: 14.2s	remaining: 16m 39s
7:	learn: 0.4915157	test: 0.5826691	best: 0.5826691 (7)	total: 15.9s	remaining: 16m 19s
8:	learn: 0.4848795	test: 0.5823041	best: 0.5823041 (8)	total: 17.7s	remaining: 16m 4s
9:	learn: 0.4829311	test: 0.5822145	best: 0.5822145 (9)	total: 19.4s	remaining: 15m 50s
10:	learn: 0.4800897	test: 0.5805919	best: 0.5805919 (10)	total: 21.2s	remaining: 15m 42s
11:	learn: 0.4770363	test: 0.5806285	be

In [362]:
# Save model
fit_model.save_model('/content/drive/MyDrive/Гилфорд Маг Диплом Розовская/New results/CB2_reg_orig_6_500-08-10')

In [363]:
# Load model
model = CatBoostRegressor()
fit_model2 = model.load_model('/content/drive/MyDrive/Гилфорд Маг Диплом Розовская/New results/CB2_reg_orig_6_500-08-10')

In [364]:
# Get predicted classes
preds = fit_model2.predict(eval_data)
preds = preds.tolist()

In [365]:
print('explained_variance_score: ', explained_variance_score(eval_labels, preds))
print('r2_score: ', r2_score(eval_labels, preds))

explained_variance_score:  0.5986234656005811
r2_score:  0.5985712592294841


In [366]:
print('get_best_iteration: ', fit_model2.get_best_iteration())
print('get_best_score: ', fit_model2.get_best_score())

get_best_iteration:  16
get_best_score:  {'learn': {'RMSE': 0.03860555741771482}, 'validation': {'RMSE': 0.578864837490677}}
